In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')


## Load data

In [11]:
dset_path = './norm_counts_AML.txt' # normalized counts
anno_path = './annotation_AML.txt' # label annotations
gene_path = './L1000_landmark_gene_list.txt' # the ~1000 most important genes 

### Note: data look as follows, which is different from the common form in ML (#samples, #feats).
# Thefore remove the first column and row and then transpose the whole matrix.
dset = pd.read_csv(dset_path, sep='\t', header=0, index_col=0) 
dset = dset.T

### Load the label
anno = []
classes = ['ALL','AML','CLL','CML']  # We consider 5 classes in total ['ALL','AML','CLL','CML','Other']
with open(anno_path) as f:
    ### Discard the header manually.
    f.readline()
    
    ### Each line consists of Dataset, GSE, Condition, Disease, Tissue, FAB, and Filename
    for line in f.readlines():
        line = line.strip()
        label = line.split('\t')[-4]  # Get the Disease label
        if label not in classes:
            label = 'Other'
        anno.append(label)
        
### Map strings to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(anno)
label_map = label_encoder.classes_
label_dict = {}
for ind, ll in enumerate(label_map):
    label_dict[ll] = ind
    
### Check whether both the dataset and labels have the same amount of samples
print(f'dset shape: {dset.shape}')
print(f'labels shape: {labels.shape}')
assert dset.shape[0] == labels.shape[0]
print(label_dict)

### Select genes
if gene_path is not None:
    gene = []
    with open(gene_path) as f:
        f.readline()  # Discard the header manually.
        for line in f.readlines():
            line = line.strip()
            gene.append(line)
    gene = set(gene)
    extract_dset = []
    column_names = []
    for g in dset.columns:
        if g in gene:
            column_names.append(g)
            extract_dset.append(dset[g])
    extract_dset = np.array(extract_dset).T

### Get the dataframe of the datasets 
dset_new = pd.DataFrame(extract_dset, columns=column_names)

dset shape: (1181, 12688)
labels shape: (1181,)
{'ALL': 0, 'AML': 1, 'CLL': 2, 'CML': 3, 'Other': 4}


In [12]:
dset_new

,TSPAN6,SCYL3,BAD,LAP3,SNX11,CASP10,CFLAR,FKBP4,RBM6,SLC25A13,...,TWF2,HOXA10,LYN,CHMP4A,POLG2,RBM15B,MRPL12,IKBKE,DUSP14,PIP4K2B
0,58.309329,947.249548,262.827054,2499.139459,1202.334653,2651.310067,12072.047943,1256.537156,6143.061467,528.909460,...,2618.094047,27.044490,11162.118590,1419.371712,657.013983,2910.446839,416.556002,1489.254207,155.963513,3570.903461
1,21.749611,882.292463,316.557942,4717.732805,1234.354732,2585.812464,11762.509547,1508.100652,4730.205872,433.416293,...,2846.626051,10.566099,12086.450188,1464.642220,546.699860,3081.562153,473.824269,1574.303273,113.613739,2651.646969
2,39.662588,956.171609,346.374400,3547.744390,1319.419694,1982.908005,12981.783605,955.769729,5514.578712,317.671363,...,2809.919788,14.878002,15052.703394,1395.778722,537.266323,3086.247744,429.547932,1685.773020,121.916948,2625.520733
3,42.981438,1099.730346,236.296965,3267.089738,1201.786894,2122.333722,12823.387819,1339.664632,4915.787019,247.913198,...,2920.213814,40.852211,13880.171091,1276.311267,490.030545,2979.086434,438.933838,1527.884354,120.928759,2574.371458
4,55.387919,1123.974430,361.452649,2470.471549,1074.910091,1971.325285,14209.841838,1187.799820,5133.584689,344.192257,...,3606.850430,53.197113,10637.570162,1955.099675,797.854393,2693.151946,417.309487,1572.719881,144.033076,2304.661934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,75.825586,592.744803,1214.049189,2091.405578,964.464658,749.761922,10597.968502,1743.204809,2898.811761,126.628719,...,7006.251785,24.722606,3869.892875,2750.391683,186.089580,5004.380572,1603.391220,1752.444913,366.096996,2050.158300
1177,120.423181,612.842102,1070.499016,1610.288157,863.259766,964.730213,11573.309476,2156.112023,2702.216238,120.631007,...,6821.360269,23.281035,3301.305437,2283.180671,233.720359,3922.194956,1730.351791,1569.026222,367.354754,2002.598708
1178,95.024053,607.111684,897.361824,2177.705191,1017.006081,913.727106,8387.670650,1562.803945,2865.468053,147.749335,...,7401.155404,24.489525,4548.941388,3086.823002,279.201370,4216.050318,1767.902967,1599.301804,267.351995,1775.212669
1179,195.564764,667.412119,1111.066101,1044.589229,813.859557,884.854435,9180.275092,1644.305893,3478.387619,126.824462,...,5447.391904,19.317114,2913.354800,1888.583563,408.026838,3241.971510,1485.192938,1311.594925,367.592873,1777.697237


In [13]:
dset_new.head()

,TSPAN6,SCYL3,BAD,LAP3,SNX11,CASP10,CFLAR,FKBP4,RBM6,SLC25A13,...,TWF2,HOXA10,LYN,CHMP4A,POLG2,RBM15B,MRPL12,IKBKE,DUSP14,PIP4K2B
0,58.309329,947.249548,262.827054,2499.139459,1202.334653,2651.310067,12072.047943,1256.537156,6143.061467,528.909460,...,2618.094047,27.044490,11162.118590,1419.371712,657.013983,2910.446839,416.556002,1489.254207,155.963513,3570.903461
1,21.749611,882.292463,316.557942,4717.732805,1234.354732,2585.812464,11762.509547,1508.100652,4730.205872,433.416293,...,2846.626051,10.566099,12086.450188,1464.642220,546.699860,3081.562153,473.824269,1574.303273,113.613739,2651.646969
2,39.662588,956.171609,346.374400,3547.744390,1319.419694,1982.908005,12981.783605,955.769729,5514.578712,317.671363,...,2809.919788,14.878002,15052.703394,1395.778722,537.266323,3086.247744,429.547932,1685.773020,121.916948,2625.520733
3,42.981438,1099.730346,236.296965,3267.089738,1201.786894,2122.333722,12823.387819,1339.664632,4915.787019,247.913198,...,2920.213814,40.852211,13880.171091,1276.311267,490.030545,2979.086434,438.933838,1527.884354,120.928759,2574.371458
4,55.387919,1123.974430,361.452649,2470.471549,1074.910091,1971.325285,14209.841838,1187.799820,5133.584689,344.192257,...,3606.850430,53.197113,10637.570162,1955.099675,797.854393,2693.151946,417.309487,1572.719881,144.033076,2304.661934


## Discretization

In [14]:
# get statistics
alpha=0.2

def discretize(data,alpha):
    alphas = [alpha, 1-alpha]  # let num_active = num_inactive 
    data_quantile =np.quantile(data, alphas, axis=0)

    # discretization given the pre-defined quantile 
    data_discrete = []
    for idx, col in enumerate(data.columns):
        discrete_col = np.digitize(data[col],data_quantile[:,idx])
        data_discrete.append(discrete_col)
    data_discrete = np.array(data_discrete).T
    return data_discrete

### The discretized dataset
dset_discrete = discretize(dset_new,alpha)

## Classification (with LinearSVM)

In [15]:
### pre-process the dataset
dset_standard = StandardScaler().fit_transform(dset)  # standardization
dset_minmax = MinMaxScaler().fit_transform(dset) # min-max scale

In [16]:
def eval_svm(data, labels, test_fraction, k):
    Train_x, Test_x, Train_y, Test_y = train_test_split(data, labels, test_size=test_fraction,
                                                        random_state=k)

    ## l1
    lsvc = LinearSVC(C=0.01, penalty="l1", dual="auto").fit(Train_x, Train_y)
    model=lsvc
    print("LinearSVC l1: test acc", model.score(Test_x,Test_y))
    model = SelectFromModel(lsvc, prefit=True)
    x_new = model.transform(data)
    print(data.shape)
    print(x_new.shape)


    ## l2
    lsvc = LinearSVC(penalty="l2").fit(Train_x, Train_y)
    model=lsvc
    print("LinearSVC l2: test acc", model.score(Test_x,Test_y))
    model = SelectFromModel(lsvc, prefit=True)
    x_new = model.transform(data)
    print(data.shape)
    print(x_new.shape)


def eval_svm_twoside(data, labels, test_fraction, k):
    Train_x, Test_x, Train_y, Test_y = train_test_split(data, labels, test_size=test_fraction,
                                                        random_state=k)

    ## l1
    lsvc = LinearSVC(C=0.01, penalty="l1", dual="auto").fit(Train_x, Train_y)
    model=lsvc
    print("LinearSVC l1: test acc", model.score(Test_x,Test_y))
    model = SelectFromModel(lsvc, prefit=True)
    x_new = model.transform(data)
    print(data.shape)
    print(x_new.shape)

    ## l1
    lsvc = LinearSVC(C=0.01, penalty="l1", dual="auto").fit(Test_x, Test_y)
    model=lsvc
    print("LinearSVC l1: test acc", model.score(Train_x,Train_y))
    model = SelectFromModel(lsvc, prefit=True)
    x_new = model.transform(data)
    print(data.shape)
    print(x_new.shape)

    ## l2
    lsvc = LinearSVC(penalty="l2").fit(Train_x, Train_y)
    model=lsvc
    print("LinearSVC l2: test acc", model.score(Test_x,Test_y))
    model = SelectFromModel(lsvc, prefit=True)
    x_new = model.transform(data)
    print(data.shape)
    print(x_new.shape)
    
    ## l2
    lsvc = LinearSVC(penalty="l2").fit(Test_x, Test_y)
    model=lsvc
    print("LinearSVC l2: test acc", model.score(Train_x,Train_y))
    model = SelectFromModel(lsvc, prefit=True)
    x_new = model.transform(data)
    print(data.shape)
    print(x_new.shape)
    
### evaluate both pre-processing schemes
seed=1000
test_fraction = 0.2

print('No preprocessing')
eval_svm_twoside(dset, labels, test_fraction, seed)

print('Standardization')
eval_svm_twoside(dset_standard, labels, test_fraction, seed)

print('Minmax Normalization')
eval_svm_twoside(dset_minmax, labels, test_fraction, seed)

No preprocessing
LinearSVC l1: test acc 0.9831223628691983
(1181, 12688)
(1181, 29)
LinearSVC l1: test acc 0.9597457627118644
(1181, 12688)
(1181, 16)
LinearSVC l2: test acc 0.9957805907172996
(1181, 12688)
(1181, 3281)
LinearSVC l2: test acc 0.9597457627118644
(1181, 12688)
(1181, 3202)
Standardization
LinearSVC l1: test acc 0.9831223628691983
(1181, 12688)
(1181, 106)
LinearSVC l1: test acc 0.9417372881355932
(1181, 12688)
(1181, 34)
LinearSVC l2: test acc 0.810126582278481
(1181, 12688)
(1181, 4936)
LinearSVC l2: test acc 0.8877118644067796
(1181, 12688)
(1181, 5176)
Minmax Normalization
LinearSVC l1: test acc 0.9535864978902954
(1181, 12688)
(1181, 10)
LinearSVC l1: test acc 0.527542372881356
(1181, 12688)
(1181, 1)
LinearSVC l2: test acc 0.9915611814345991
(1181, 12688)
(1181, 5149)
LinearSVC l2: test acc 0.9745762711864406
(1181, 12688)
(1181, 5146)


## Discretized data classification

In [17]:
### evaluate both pre-processing scheme
seed=1000
test_fraction = 0.2

### loop over different alpha values
alpha_list = np.linspace(0.1,0.9,17)
for alpha in alpha_list:
    print('='*30)
    print('alpha=%f'%alpha)
    dset_discrete = discretize(dset_new,alpha)
    
    ### Treat the as ordinal attributes 
    print('ordinal') 
    eval_svm(dset_discrete, labels, test_fraction, seed)

    ### Treat as categorical attributes
    print('categorical')
    dset_onehot = OneHotEncoder().fit_transform(dset_discrete)
    eval_svm(dset_onehot, labels, test_fraction, seed)

alpha=0.100000
ordinal
LinearSVC l1: test acc 0.9662447257383966
(1181, 958)
(1181, 220)
LinearSVC l2: test acc 0.9915611814345991
(1181, 958)
(1181, 401)
categorical
LinearSVC l1: test acc 0.9324894514767933
(1181, 2851)
(1181, 155)
LinearSVC l2: test acc 0.9915611814345991
(1181, 2851)
(1181, 1201)
alpha=0.150000
ordinal
LinearSVC l1: test acc 0.9831223628691983
(1181, 958)
(1181, 247)
LinearSVC l2: test acc 0.9915611814345991
(1181, 958)
(1181, 411)
categorical
LinearSVC l1: test acc 0.9620253164556962
(1181, 2858)
(1181, 107)
LinearSVC l2: test acc 0.9915611814345991
(1181, 2858)
(1181, 1225)
alpha=0.200000
ordinal
LinearSVC l1: test acc 0.9831223628691983
(1181, 958)
(1181, 263)
LinearSVC l2: test acc 0.9957805907172996
(1181, 958)
(1181, 424)
categorical
LinearSVC l1: test acc 0.9662447257383966
(1181, 2859)
(1181, 89)
LinearSVC l2: test acc 0.9915611814345991
(1181, 2859)
(1181, 1264)
alpha=0.250000
ordinal
LinearSVC l1: test acc 0.9831223628691983
(1181, 958)
(1181, 249)
Linear

In [ ]:

alpha_list = [0.2]
for alpha in alpha_list:
    print('='*30)
    print('alpha=%f'%alpha)
    dset_discrete = discretize(dset_new,alpha)

    
    ### Treat as ordinal attributes
    print('ordinal')
    eval_svm_twoside(dset_discrete, labels, test_fraction, seed)
    
    ### Treat as categorical attributes
    print('categorical')
    dset_onehot = OneHotEncoder().fit_transform(dset_discrete)
    eval_svm_twoside(dset_onehot, labels, test_fraction, seed)

alpha=0.200000
ordinal
LinearSVC l1: test acc 0.9831223628691983
(1181, 958)
(1181, 263)
LinearSVC l1: test acc 0.9057203389830508
(1181, 958)
(1181, 66)
LinearSVC l2: test acc 0.9957805907172996
(1181, 958)
(1181, 424)
LinearSVC l2: test acc 0.9639830508474576
(1181, 958)
(1181, 406)
categorical
LinearSVC l1: test acc 0.9662447257383966
(1181, 2859)
(1181, 89)
LinearSVC l1: test acc 0.7574152542372882
(1181, 2859)
(1181, 7)
LinearSVC l2: test acc 0.9915611814345991
(1181, 2859)
(1181, 1264)
LinearSVC l2: test acc 0.9661016949152542
(1181, 2859)
(1181, 1260)
